In [1]:
from cassandra.cluster import Cluster, PlainTextAuthProvider
from cassandra.cluster import (
    NoHostAvailable,
    OperationTimedOut
)
import os, sys
import pandas as pd, numpy as np
import uuid
CASS_HOST = os.getenv('CASS_HOST') or 'localhost'
CASS_USER = os.getenv('CASS_USER') or 'cassandra'
CASS_PASS = os.getenv('CASS_PASS') or 'changeme'

In [ ]:
class cassDB():
    def __init__(self,user,pwd,host,db):
        self.auth_provider = PlainTextAuthProvider(
            username=user,
            password = pwd
        )
        self.host = host
        self.db = db
        self.cluster= None 
        self.session = None
        # self.connect()
    def connect(self):
        # logger.info( " [x] Initializating DB Connection to Cassandra @ {}".format(self.host))
        self.cluster = Cluster(
            [self.host],
            auth_provider=self.auth_provider
        )
        # logger.info("  [x] Establishing cluster session")
        self.session = self.cluster.connect()
        # logger.info("  [x] Connecting to database: {}".format(self.db))
        self.session.set_keyspace(self.db)
        self.session.execute('USE {};'.format(self.db))
        # logger.info( " [x] Connected to Cassandra Database!")

    def rows_to_json(self,rows):
        result = []
        for row in rows:
            d = row._asdict()
            for k,v in d.items():
                if isinstance(v,uuid.UUID):
                    d[k] = str(v)
            result.append(d)
        return result

    def parse_spells(self,rows):
        #convert data back to JSON/py dictionary
        for i in range(len(rows)):
            row = rows[i]
            dmg_dict = {}
            dmg = row['dmg']
            for level,dmgmap in dmg.items():
                dmg_dict[level] = {}
                for dmgtype,die_list in dmgmap.items():
                    dmg_dict[level][dmgtype] = []
                    for die in die_list:
                        dmg_dict[level][dmgtype].append(dict(die))
            upcast = dict(row['upcast'])
            curr_dict = rows[i].copy()
            curr_dict['upcast'] = upcast
            curr_dict['dmg']=dmg_dict
            rows[i] = curr_dict.copy()
        return rows

    def parse_weapons(self,rows):
        #convert data back to JSON/py dictionary
        for i in range(len(rows)):
            row = rows[i]
            dmg_dict = {}
            dmg = row['dmg']
            for hands,dmgmap in dmg.items():
                dmg_dict[hands]={}
                for dmgtype,dice in dmgmap.items():
                    dmg_dict[hands][dmgtype] = dict(dice)
            curr_dict = row.copy()
            curr_dict['dmg'] = dmg_dict
            rows[i] = curr_dict.copy()
        return rows

    ## parse out class data
    def parse_class(self,rows):
        pass

    ## parse out spellslots?
    def parse_spellslots(self,rows):
        for i in range(len(rows)):
            row = rows[i]
            row['spellslots'] = dict(row['spellslots'])
            rows[i] = row
        return rows

    def parse_char(self,rows):
        for i in range(len(rows)):
            row = rows[i]
            row['spellslots'] = dict(row['spellslots'])
            row['equipped'] = dict(row['equipped'])
        return rows


    def _rows_to_json(self,rows):
        result = []
        for i in range(len(rows)):
            row = rows[i]
            d = row._asdict()
            for k,v in d.items():
                if isinstance(v,uuid.UUID):
                    d[k] = str(v)
            result.append(d)
        return result

    def exec_query(self,s):
        result = {
            'query':s,
            'success':False
        }
        try: 
            response = self.session.execute(s)
        except NoHostAvailable:
            self.connect()
        except OperationTimedOut:
            self.connect()
        finally:
            response = self.session.execute(s)
            result['success'] = True
        # logger.info("  [x] DB on_request: query succeeded: {}".format(s))
        l = response.all()
        result['success']= True
        print(l)
        if len(l) > 0:
            result['rows'] = self.rows_to_json(l)
        if "FROM spells" in s:
            result['rows'] = self.parse_spells(result['rows'])
        if "FROM weapons" in s:
            result['rows'] = self.parse_weapons(result['rows'])
        if "FROM char_table" in s:
            print(result['rows'])
            result['rows'] = self.parse_char(result['rows'])
        #placeholder for other parsing
        # if "FROM classes" in s:
        #     pass
        return result


In [23]:
db = cassDB(CASS_USER,CASS_PASS,CASS_HOST,'dndio')
db.connect()
db.exec_query('use dndio;')

[]


{'query': 'use dndio;', 'success': True}

In [24]:
q = "SELECT * FROM char_table WHERE char_id='chorky#8402' and campaign_id='abcdef';"
res = db.exec_query(q)
res
# display(res,dict(res['rows'][0]['equipped']),dict(res['rows'][0]['spellslots']))

[Row(campaign_id='abcdef', char_id='chorky#8402', ac=18, align='Chaotic Neutral', armor=None, background=None, cha=19, char_class='Warlock', char_name='Bahlok', con=15, curr_hp=43, dex=18, equipped=OrderedMapSerializedKey([('weapon', 'dagger')]), int=11, level=7, max_hp=50, skills=['intimidation', 'sleight of hand', 'persuasion', 'performance'], spells=['Eldritch Blast'], spellslots=OrderedMapSerializedKey([(4, 2)]), str=10, weapons=['dagger', 'light crossbow'], wis=9)]
[{'campaign_id': 'abcdef', 'char_id': 'chorky#8402', 'ac': 18, 'align': 'Chaotic Neutral', 'armor': None, 'background': None, 'cha': 19, 'char_class': 'Warlock', 'char_name': 'Bahlok', 'con': 15, 'curr_hp': 43, 'dex': 18, 'equipped': OrderedMapSerializedKey([('weapon', 'dagger')]), 'int': 11, 'level': 7, 'max_hp': 50, 'skills': ['intimidation', 'sleight of hand', 'persuasion', 'performance'], 'spells': ['Eldritch Blast'], 'spellslots': OrderedMapSerializedKey([(4, 2)]), 'str': 10, 'weapons': ['dagger', 'light crossbow']

{'query': "SELECT * FROM char_table WHERE char_id='chorky#8402' and campaign_id='abcdef';",
 'success': True,
 'rows': [{'campaign_id': 'abcdef',
   'char_id': 'chorky#8402',
   'ac': 18,
   'align': 'Chaotic Neutral',
   'armor': None,
   'background': None,
   'cha': 19,
   'char_class': 'Warlock',
   'char_name': 'Bahlok',
   'con': 15,
   'curr_hp': 43,
   'dex': 18,
   'equipped': {'weapon': 'dagger'},
   'int': 11,
   'level': 7,
   'max_hp': 50,
   'skills': ['intimidation', 'sleight of hand', 'persuasion', 'performance'],
   'spells': ['Eldritch Blast'],
   'spellslots': {4: 2},
   'str': 10,
   'weapons': ['dagger', 'light crossbow'],
   'wis': 9}]}

In [48]:
skill = "arcana"

q = "SELECT * FROM char_table WHERE user_id='chorky#8402' and campaign_id='abcdef';"

res = db.exec_query(q)
res

{'query': "SELECT * FROM char_table WHERE user_id='chorky#8402' and campaign_id='abcdef';",
 'success': True,
 'rows': [{'campaign_id': 'abcdef',
   'user_id': 'chorky#8402',
   'ac': 15,
   'align': 'Chaotic Neutral',
   'armor': None,
   'background': None,
   'char_class': 'Warlock',
   'char_name': 'Bahlok',
   'chr': 15,
   'con': 15,
   'curr_hp': 40,
   'dex': 18,
   'equipped': OrderedMapSerializedKey([('weapon', 'dagger')]),
   'hit_dice': None,
   'int': 15,
   'level': 7,
   'max_hp': 53,
   'prof_bonus': None,
   'saving_throws': None,
   'skills': ['intimidation', 'sleight of hand', 'persuasion', 'performance'],
   'spell_dc': 16,
   'spells': ['Eldritch Blast'],
   'spellslots': {4: 2},
   'str': 15,
   'weapons': ['dagger', 'hand crossbow'],
   'wis': 15}]}

In [49]:
chr_data = res['rows'][0]
skills = chr_data['skills']
skills

['intimidation', 'sleight of hand', 'persuasion', 'performance']

In [50]:
q = "SELECT prof_bonus FROM classes WHERE class_id='{}'".format(chr_data['char_class']+'-'+str(chr_data['level']))

chr_class = db.exec_query(q)

chr_class

{'query': "SELECT prof_bonus FROM classes WHERE class_id='Warlock-7'",
 'success': True,
 'rows': [{'prof_bonus': 3}]}

In [51]:
q = "SELECT * from skills where skill='{}'".format('sleight of hand')

skill_data = db.exec_query(q) #['rows'][0]

In [52]:
skill_data['rows'][0]['modifier'].lower()

'dex'

In [53]:
chr_data[skill_data['rows'][0]['modifier'].lower()]

18

In [54]:
chr_mod = (chr_data[skill_data['rows'][0]['modifier'].lower()] - 10)//2

In [55]:
chr_mod

4

In [87]:
import random

roll = [random.randint(1,20)]

In [88]:
roll

[9]

In [89]:
[r+chr_mod for r in roll]

[13]

In [90]:
## roll spell damage...
char_query = """SELECT level, spells, spell_dc, spellslots FROM char_table WHERE user_id='chorky#8402' and campaign_id='abcdef'"""
res = db.exec_query(char_query)
res

{'query': "SELECT level, spells, spell_dc, spellslots FROM char_table WHERE user_id='chorky#8402' and campaign_id='abcdef'",
 'success': True,
 'rows': [{'level': 7,
   'spells': ['Eldritch Blast'],
   'spell_dc': 16,
   'spellslots': {4: 2}}]}

In [60]:
spell = db.exec_query("""SELECT * FROM spells WHERE name='Eldritch Blast' allow filtering;""")

In [61]:
spell = spell['rows'][0]
spell

{'name': 'Eldritch Blast',
 'addspellmod': False,
 'char_classes': ['Warlock'],
 'components': ' V, S ',
 'conc': False,
 'description': None,
 'dmg': {1: {'Force': [{1: 10}]}},
 'duration': 'Instantaneous',
 'hardmod': 0,
 'level': 0,
 'mult': 1,
 'range': ' 120 feet ',
 'save': 'nan',
 'save_data': 'nan',
 'save_success': "Source: Player's Handbook\n Evocation cantrip\n Casting Time:\n 1 action\n Range:\n 120 feet\n Components:\n V, S\n Duration:\n Instantaneous\n A beam of crackling energy streaks toward a creature within range. Make a ranged spell attack against the target. On a hit, the target takes 1d10 force damage.\n At Higher Levels.\n The spell creates more than one beam when you reach higher levels: two beams at 5th level, three beams at 11th level, and four beams at 17th level. You can direct the beams at the same target or at different ones. Make a separate attack roll for each beam.\n Spell Lists.\n Warlock",
 'time': ' 1 action ',
 'upcast': {}}

In [62]:
spell['name'] in res['rows'][0]['spells']

True

In [63]:
res['rows'][0]['spellslots']

{4: 2}

In [91]:
mult = spell['mult']
hard_mod = spell['hardmod']
upcast = dict(spell['upcast'])
dmg = spell['dmg']
char_level = res['rows'][0]['level']
slot_lvl = 4
dmg

{1: {'Force': [{1: 10}]}}

In [92]:
if len(upcast.keys()) > 0:
    upcast_lvl = list(upcast.keys())[0]
else:
    upcast_lvl=21
if upcast_lvl <= slot_lvl:
    upcast_dice = slot_lvl - upcast_lvl + 1
else:
    upcast_dice = 0
lvls = [x for x in dmg.keys() if x <= char_level]
lvls.sort()
target_dmg = dmg.get(lvls[-1])


In [93]:
results = {}
mod_results = {}
for k,v in target_dmg.items():
    results[k] = []
    mod_results[k] = []
    for l,w in v[0].items(): # this may need to change...
        for i in range(upcast_dice+(l*mult)):
            roll = random.randint(1,w)
            results[k].append(roll)
            mod_results[k].append(roll+hard_mod)

display(results, mod_results)

{'Force': [7]}

{'Force': [7]}

In [67]:
char_query = """SELECT * FROM char_table WHERE user_id='chorky#8402' and campaign_id='abcdef';"""
char_data = db.exec_query(char_query)['rows'][0]

In [68]:
char_data

{'campaign_id': 'abcdef',
 'user_id': 'chorky#8402',
 'ac': 15,
 'align': 'Chaotic Neutral',
 'armor': None,
 'background': None,
 'char_class': 'Warlock',
 'char_name': 'Bahlok',
 'chr': 15,
 'con': 15,
 'curr_hp': 40,
 'dex': 18,
 'equipped': OrderedMapSerializedKey([('weapon', 'dagger')]),
 'hit_dice': None,
 'int': 15,
 'level': 7,
 'max_hp': 53,
 'prof_bonus': None,
 'saving_throws': None,
 'skills': ['intimidation', 'sleight of hand', 'persuasion', 'performance'],
 'spell_dc': 16,
 'spells': ['Eldritch Blast'],
 'spellslots': {4: 2},
 'str': 15,
 'weapons': ['dagger', 'hand crossbow'],
 'wis': 15}

In [69]:
wep_query = """SELECT * FROM weapons WHERE name='dagger';"""
wep_data = db.exec_query(wep_query)['rows'][0]

In [70]:
wep_data

{'name': 'dagger',
 'cost': 2,
 'cost_unit': 'gp',
 'dmg': {'1hnd': {'piercing': {1: 4}}},
 'long_range': 60,
 'mod': ['STR', 'DEX'],
 'normal_range': 20,
 'props': ['Finesse', 'light', 'thrown (20/60)'],
 'subtype': 'simple',
 'type': 'Melee',
 'weight': 1.0}

In [71]:
prof_query = """SELECT proficiencies['weapon_class'] AS weapon_class,
proficiencies['weapons'] AS weapons FROM class_start WHERE char_class='{}';""".format(char_data['char_class'])
prof_data = db.exec_query(prof_query)['rows'][0]

prof_data['weapons'] = [] #need to fix this in source data...
prof_data

{'weapon_class': ['simple'], 'weapons': []}

In [72]:
char_data['char_class']

'Warlock'

In [73]:
class_id = char_data['char_class']+'-'+str(char_data['level'])
print(class_id)
class_query = """SELECT * from classes WHERE class_id = '{}';""".format(class_id)
class_data = db.exec_query(class_query)['rows'][0]
class_data
##need to fix the spellslots and the class_specifics for class information returns in the database interface...

Warlock-7


{'class_id': 'Warlock-7',
 'field_1_': 'Warlock',
 'class_specific': OrderedMapSerializedKey([('cantrips_known', 3), ('invocations_known', 4), ('spells_known', 8)]),
 'features': None,
 'level': 7,
 'prof_bonus': 3,
 'spellslots': OrderedMapSerializedKey([(4, 2)])}

In [74]:
prof_bonus = class_data['prof_bonus']

In [75]:
rolls = [random.randint(1,20) for i in range(2)]

In [76]:
rolls

[14, 13]

In [77]:
mod_rolls = rolls.copy()
mod_rolls

[14, 13]

In [78]:
prof_data

{'weapon_class': ['simple'], 'weapons': []}

In [79]:
if wep_data['name'] in prof_data['weapons'] or wep_data['subtype'] in prof_data['weapon_class']:
    mod_rolls = [m+prof_bonus for m in mod_rolls]
mod_rolls

[17, 16]

In [80]:
char_data['dex']

18

In [81]:
add_mods = []
for m in wep_data['mod']:
    add_mods.append((m,(char_data[m.lower()]-10)//2))
add_mod = max(add_mods,key=lambda x: x[1])
add_mod

('DEX', 4)

In [82]:
add_mods

[('STR', 2), ('DEX', 4)]

In [83]:
mod_rolls = [m+add_mod[1] for m in mod_rolls]
mod_rolls

[21, 20]

In [84]:
total = max(mod_rolls)

In [85]:
total

21

In [86]:
{
    'rolls':rolls,
    'mod_rolls':mod_rolls,
    'modifiers':[add_mod,('prof_bonus',prof_bonus)],
    'total':total
}

{'rolls': [14, 13],
 'mod_rolls': [21, 20],
 'modifiers': [('DEX', 4), ('prof_bonus', 3)],
 'total': 21}